In [1]:
# Mrityunjay kumar #
# Loss Visualization with TensorFlow.
# This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)

In [2]:
import tensorflow as tf
import numpy
tf.compat.v1.disable_eager_execution()

# Import MINST data
import input_data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0129 11:54:33.148290 4642252224 deprecation.py:323] From <ipython-input-2-5e2f98863a6b>:7: read_data_sets (from input_data) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
W0129 11:54:33.149149 4642252224 deprecation.py:323] From /Users/mrityunjay/Desktop/Mrityunjay_Kumar/Tensor_NN/input_data.py:297: _maybe_download (from input_data) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0129 11:54:33.150002 4642252224 deprecation.py:323] From /Users/mrityunjay/Desktop/Mrityunjay_Kumar/Tensor_NN/input_data.py:299: _extr

Extracting MNIST/train-images-idx3-ubyte.gz


W0129 11:54:33.495723 4642252224 deprecation.py:323] From /Users/mrityunjay/Desktop/Mrityunjay_Kumar/Tensor_NN/input_data.py:304: _extract_labels (from input_data) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0129 11:54:33.498906 4642252224 deprecation.py:323] From /Users/mrityunjay/Desktop/Mrityunjay_Kumar/Tensor_NN/input_data.py:112: _dense_to_one_hot (from input_data) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0129 11:54:33.572011 4642252224 deprecation.py:323] From /Users/mrityunjay/Desktop/Mrityunjay_Kumar/Tensor_NN/input_data.py:328: _DataSet.__init__ (from input_data) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Use Logistic Regression from our previous example
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.compat.v1.placeholder("float", [None, 784], name='x') # mnist data image of shape 28*28=784
y = tf.compat.v1.placeholder("float", [None, 10], name='y') # 0-9 digits recognition => 10 classes

# Create model

# Set model weights
W = tf.compat.v1.Variable(tf.compat.v1.zeros([784, 10]), name="weights")
b = tf.compat.v1.Variable(tf.compat.v1.zeros([10]), name="bias")

# Construct model
activation = tf.compat.v1.nn.softmax(tf.compat.v1.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = -tf.compat.v1.reduce_sum(y*tf.compat.v1.log(activation)) # Cross entropy
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost) # Gradient Descent

# Initializing the variables
init = tf.compat.v1.initialize_all_variables()

W0129 11:54:34.416870 4642252224 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
# Create a summary to monitor cost function
tf.compat.v1.summary.scalar ("loss", cost)

# Merge all summaries to a single operator
merged_summary_op = tf.compat.v1.summary.merge_all ()

In [5]:
# Launch the graph
with tf.compat.v1.Session() as sess:
    sess.run(init)

    # Set logs writer into folder /tmp/tensorflow_logs
    summary_writer = tf.compat.v1.summary.FileWriter('/tmp/tensorflow_logs', graph_def=sess.graph_def)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs at every iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, epoch*total_batch + i)
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.compat.v1.equal(tf.compat.v1.argmax(activation, 1), tf.compat.v1.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.compat.v1.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

W0129 11:54:34.472738 4642252224 writer.py:199] Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.


Epoch: 0001 cost= 30.391744385
Epoch: 0002 cost= 21.960723222
Epoch: 0003 cost= 20.997861068
Epoch: 0004 cost= 20.501458651
Epoch: 0005 cost= 20.243254188
Epoch: 0006 cost= 19.918340168
Epoch: 0007 cost= 19.742126565
Epoch: 0008 cost= 19.500379227
Epoch: 0009 cost= 19.387135324
Epoch: 0010 cost= 19.125067529
Optimization Finished!
Accuracy: 0.8984


### Run the command line
```
tensorboard --logdir=/tmp/tensorflow_logs
```

### Open http://localhost:6006/ into your web browser

In [6]:
# Loss per minibatch step